In [1]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import sys
sys.path.append('../.') 
from src.graph.create_osmnx_graph import OSMnxGraph
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import networkx as nx
%reload_ext autoreload
from src.baseline_models.GCN.supervised_node_classification import SupervisedNodeClassificationGNN
from src.baseline_models.GCN.gcn import GCNModel
import torch
from sklearn.model_selection import train_test_split
import pytorch_lightning as pl
from src.baseline_models.GCN.GraphData import GraphData
import warnings
warnings.filterwarnings('ignore')
from shapely.geometry import Point

/Users/jjaniak/anaconda3/envs/studia_projekt/lib/python3.8/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/4k/8js3jm9n457cm4_v631zyrkw0000gn/T/ipykernel_67880/2805524801.py:2: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, th

# Wczytanie danych

In [2]:
df_accidents = pd.read_csv("../data/wypadki-pl/accidents.csv")
df_accidents.drop(df_accidents[(df_accidents['mie_nazwa'] != 'Warszawa')].index, inplace=True)
df_accidents.drop(columns='uczestnicy', inplace=True)
geometry = [Point(xy) for xy in zip(df_accidents['wsp_gps_x'], df_accidents['wsp_gps_y'])]
gdf_events = gpd.GeoDataFrame(df_accidents, geometry=geometry)
gdf_events.drop(columns=['wsp_gps_x', 'wsp_gps_y'], inplace=True)
G =ox.graph.graph_from_place("Warsaw, Poland", network_type="drive")
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)
gdf_nodes

,y,x,street_count,highway,ref,geometry
osmid,,,,,,
26063726,52.152787,21.017454,5,NaN,NaN,POINT (21.01745 52.15279)
26063848,52.157570,20.991392,4,NaN,NaN,POINT (20.99139 52.15757)
26063857,52.166411,20.992301,3,NaN,NaN,POINT (20.99230 52.16641)
26063923,52.170803,20.992247,3,NaN,NaN,POINT (20.99225 52.17080)
26083887,52.176026,20.995141,3,NaN,NaN,POINT (20.99514 52.17603)
...,...,...,...,...,...,...
11707394732,52.294628,21.022652,1,NaN,NaN,POINT (21.02265 52.29463)
11711719556,52.297432,21.043383,3,NaN,NaN,POINT (21.04338 52.29743)
11721637528,52.297728,21.046037,3,NaN,NaN,POINT (21.04604 52.29773)


In [3]:
gdf_events

,id_w_czas,czas_zdarzenia,woj_nazwa,pow_nazwa,gmi_nazwa,mie_nazwa,opis_zdarzenia,zdarzenie_id,id_systemu_zr,year,month,day,geometry
10244,2017-01-08,03:00,MAZOWIECKIE,Warszawa,Warszawa,Warszawa,Zderzenie pojazdów boczne,182100899,105328733,2017,1,8,POINT (20.99264 52.23186)
10245,2017-01-09,17:10,MAZOWIECKIE,Warszawa,Warszawa,Warszawa,Najechanie na pieszego,182100981,105329715,2017,1,9,POINT (20.94389 52.23442)
10246,2017-01-06,18:05,MAZOWIECKIE,Warszawa,Warszawa,Warszawa,Najechanie na pieszego,182104695,105338547,2017,1,6,POINT (21.04167 52.29258)
10247,2017-01-12,15:10,MAZOWIECKIE,Warszawa,Warszawa,Warszawa,Najechanie na pieszego,182105385,105341688,2017,1,12,POINT (20.91231 52.22606)
10248,2017-01-18,07:10,MAZOWIECKIE,Warszawa,Warszawa,Warszawa,Najechanie na pieszego,182108081,105350682,2017,1,18,POINT (20.96739 52.24478)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19893,2018-09-21,03:43,MAZOWIECKIE,Warszawa,Warszawa,Warszawa,Zderzenie pojazdów boczne,186601923,107299653,2018,9,21,POINT (20.96372 52.18850)
19894,2018-10-23,20:30,MAZOWIECKIE,Warszawa,Warszawa,Warszawa,Najechanie na pieszego,186638270,107395682,2018,10,23,POINT (20.99289 52.19678)
19895,2018-08-14,16:50,MAZOWIECKIE,Warszawa,Warszawa,Warszawa,Najechanie na pieszego,186754851,107182443,2018,8,14,POINT (20.97250 52.19344)
19896,2018-10-18,13:00,MAZOWIECKIE,Warszawa,Warszawa,Warszawa,Najechanie na pieszego,186776202,107387087,2018,10,18,POINT (21.03694 52.20669)


In [4]:
gdf_edges

osmid  \
u           v           key                                                    
26063726    31717945    0                                            4895484   
            29273144    0    [1014952979, 1014952978, 1007148203, 148749604]   
26063848    1391593855  0       [13982288, 125098266, 896320940, 1203516007]   
            258495263   0                                           23853023   
            26083906    0                                          715903941   
...                                                                      ...   
11711719556 10186274275 0                                         1237664862   
11721637528 10186274353 0                                          995022220   
            348945829   0                                         1005669962   
            10186274320 0                                         1005669962   
11730365883 7051681453  0                                         1202782555   

                             oneway   lanes                        name  \
u           v           key                                               
26063726    31717945    0      True       4                    Poleczki   
            29273144    0      True  [3, 4]                    Puławska   
26063848    1391593855  0     False       2  [Nowy Służewiec, Wyczółki]   
            258495263   0     False       2                     Zatorze   
            26083906    0     False     NaN                    Wyczółki   
...                             ...     ...                         ...   
11711719556 10186274275 0     False     NaN                         NaN   
11721637528 10186274353 0     False     NaN                         NaN   
            348945829   0     False       2                Krasiczyńska   
            10186274320 0     False       2                Krasiczyńska   
11730365883 7051681453  0      True       1             Edwarda Habicha   

                                   highway       reversed   length maxspeed  \
u           v           key                                                   
26063726    31717945    0        secondary          False   25.021      NaN   
            29273144    0          primary          False  336.457       60   
26063848    1391593855  0      residential  [False, True]  996.078      NaN   
            258495263   0         tertiary          False  353.996      NaN   
            26083906    0      residential          False   66.590      NaN   
...                                    ...            ...      ...      ...   
11711719556 10186274275 0    living_street           True    8.899       20   
11721637528 10186274353 0    living_street           True  184.729       20   
            348945829   0      residential          False   48.408       30   
            10186274320 0      residential           True   48.963       30   
11730365883 7051681453  0      residential          False  265.332       30   

                                                                      geometry  \
u           v           key                                                      
26063726    31717945    0    LINESTRING (21.01745 52.15279, 21.01780 52.15271)   
            29273144    0    LINESTRING (21.01745 52.15279, 21.01745 52.152...   
26063848    1391593855  0    LINESTRING (20.99139 52.15757, 20.99129 52.157...   
            258495263   0    LINESTRING (20.99139 52.15757, 20.99127 52.157...   
            26083906    0    LINESTRING (20.99139 52.15757, 20.99147 52.157...   
...                                                                        ...   
11711719556 10186274275 0    LINESTRING (21.04338 52.29743, 21.04335 52.297...   
11721637528 10186274353 0    LINESTRING (21.04604 52.29773, 21.04589 52.297...   
            348945829   0    LINESTRING (21.04604 52.29773, 21.04596 52.297...   
            10186274320 0    LINESTRING (21.04604 52.29773, 21.04606 52.297...   
11730365883 7051681453  0    LINESTRING (2

## Informacje o przetwarzaniu cech

### Krawędzie
Wszystkie zgromadzone cechy potraktowano analogicznie jak w pracy magisterskiej z road networks za wyjątkiem atrybutów: psv, service, busway, bicycle, cycleway i surface, które nie pojawiły się w zestawieniu (po prostu nie było ich po pobraniu przez osmnx).

Preprocessing obejmował:
* lanes - dodanie domyślnej wartości 2
* maxspeed - dodanie domyślnej wartości 50
* width - dodanie domyślnej wartości 2.0
* pozostałe - dodanie wartości "unspecified"


### Wierzchołki

Preprocessing obejmował rozbicie CountVectorizerem wartości z kolumny "highway", "street_count" pozostawiono jako wartość numeryczną bez zmian



W obu przypadkach (krawędzie i wierzchołki) usunięto kolumnę "ref".


# Agregacja do wierzchołków

## 1. Tworzenie grafu i statystyki

In [13]:
graph_embedder = OSMnxGraph(gdf_events, gdf_nodes, gdf_edges)

graph_data = graph_embedder.create_graph(element_type='node', aggregation_method='mean', normalize_y=False)
graph_data

Data(x=[19349, 8], edge_index=[2, 44140], y=[19349], street_count=[19349], highway=[19349], ref=[19349], accidents_count=[19349], osmid=[44140], oneway=[44140], lanes=[44140], name=[44140], edge_highway=[44140], reversed=[44140], length=[44140], maxspeed=[44140], geometry=[44140], access=[44140], junction=[44140], bridge=[44140], tunnel=[44140], edge_ref=[44140], width=[44140], crs=epsg:4326)

In [15]:
graph_data.y

tensor([1, 0, 0,  ..., 0, 0, 0])

In [16]:
graph_embedder.show_statistics()

{'Nodes': 19349,
 'Edges': 44140,
 'Nodes dim': 8,
 'Nodes class': 2,
 'Directed': True,
 'Graph density [%]': 0.012}

In [17]:
graph_data.num_features

8

In [18]:
len(graph_data.y.unique())

2

Poniżej znajduje się wywołanie metody do pozyskiwania cech wierzchołków.

In [19]:
features = graph_embedder.get_node_attrs()
features

,street_count,crossing,mini_roundabout,motorway_junction,speed_camera,traffic_signals,turning_circle,turning_loop
osmid,,,,,,,,
26063726,5,0,0,0,0,0,0,0
26063848,4,0,0,0,0,0,0,0
26063857,3,0,0,0,0,0,0,0
26063923,3,0,0,0,0,0,0,0
26083887,3,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
11707394732,1,0,0,0,0,0,0,0
11711719556,3,0,0,0,0,0,0,0
11721637528,3,0,0,0,0,0,0,0


## 2. Uczenie i testowanie modelu - Supervised node classification

In [20]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_features = graph_data.num_features
hidden_dim = 256
out_dim = 128
num_classes = len(graph_data.y.unique())

gnn = GCNModel(in_dim=num_features, hidden_dim=hidden_dim, out_dim=out_dim)

model = SupervisedNodeClassificationGNN(gnn=gnn, emb_dim=out_dim, num_classes=num_classes, lr=0.0001)

In [21]:

nodes_labels = {'node': features.index.to_numpy(), 'label': graph_data.y.cpu().numpy()}
df_to_split = pd.DataFrame(nodes_labels)

df_train, df_test = train_test_split(df_to_split, test_size=0.2, random_state=42, stratify=df_to_split['label'])
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=42, stratify=df_test['label'])

train_nodes = df_train['node']
val_nodes = df_val['node']
test_nodes = df_test['node']

train_mask = []   
val_mask = []
test_mask = []

for i in range(len(graph_data.y)):

  if i in train_nodes:
    train_mask.append(True)
    val_mask.append(False)
    test_mask.append(False)
  elif i in val_nodes:
    train_mask.append(False)
    val_mask.append(True)
    test_mask.append(False)
  elif i in test_nodes:
    train_mask.append(False)
    val_mask.append(False)
    test_mask.append(True)

graph_data.train_mask = torch.tensor(train_mask).cpu()
graph_data.val_mask = torch.tensor(val_mask).cpu()
graph_data.test_mask = torch.tensor(test_mask).cpu()

In [ ]:

trainer = pl.Trainer(max_epochs=50)
datamodule = GraphData([graph_data])
trainer.fit(model=model, datamodule=datamodule)

In [121]:
trainer.test(model=model, datamodule=datamodule, verbose=False)

Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 15.46it/s]


[{'step': 50.0,
  'test/auc_weighted': 0.4092303216457367,
  'test/f1_weighted': 0.7625361680984497,
  'test/precision_weighted': 0.7003365159034729,
  'test/recall_weighted': 0.8368611335754395,
  'test/accuracy_weighted': 0.8368611335754395}]

In [122]:
test_auc = trainer.test(model=model, datamodule=datamodule, verbose=False)[0]["test/auc_weighted"]

z, y, y_pred = trainer.predict(model=model, datamodule=datamodule)[0]

print(f'AUC test = {test_auc * 100.:.2f}[%]')

Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 23.22it/s]
AUC test = 40.92[%]


In [123]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [124]:
print(y[test_mask].numpy())

[0 0 0 ... 0 1 0]


# Agregacja do krawędzi

## 1. Tworzenie grafu i statystyki

In [125]:
graph_embedder = OSMnxGraph(gdf_events, gdf_nodes, gdf_edges)

graph_data = graph_embedder.create_graph(aggregation_type='edge')
graph_data

Data(x=[44177, 36], edge_index=[2, 44177], y=[44177], street_count=[19365], highway=[19365], ref=[19365], accidents_count=[19365], osmid=[44177], oneway=[44177], lanes=[44177], name=[44177], edge_highway=[44177], reversed=[44177], length=[44177], maxspeed=[44177], geometry=[44177], access=[44177], junction=[44177], bridge=[44177], tunnel=[44177], edge_ref=[44177], width=[44177], edge_accidents_count=[44177], crs=epsg:4326)

In [126]:
graph_embedder.show_statistics()

{'Nodes': 44177,
 'Edges': 44177,
 'Nodes dim': 36,
 'Nodes class': 2,
 'Directed': True,
 'Graph density [%]': 0.002}

In [127]:
features = graph_embedder.get_edge_attrs()

## 2. Uczenie i testowanie modelu - Supervised node classification

In [ ]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_features = graph_data.num_features
hidden_dim = 256
out_dim = 128
num_classes = len(graph_data.y.unique())

gnn = GCNModel(in_dim=num_features, hidden_dim=hidden_dim, out_dim=out_dim)

model = SupervisedNodeClassificationGNN(gnn=gnn, emb_dim=out_dim, num_classes=num_classes, lr=0.0001)

nodes_labels = {'node': features.index.to_numpy(), 'label': graph_data.y.cpu().numpy()}
df_to_split = pd.DataFrame(nodes_labels)

df_train, df_test = train_test_split(df_to_split, test_size=0.2, random_state=42, stratify=df_to_split['label'])
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=42, stratify=df_test['label'])

train_nodes = df_train['node']
val_nodes = df_val['node']
test_nodes = df_test['node']

train_mask = []   
val_mask = []
test_mask = []

for i in range(len(graph_data.y)):

  if i in train_nodes:
    train_mask.append(True)
    val_mask.append(False)
    test_mask.append(False)
  elif i in val_nodes:
    train_mask.append(False)
    val_mask.append(True)
    test_mask.append(False)
  elif i in test_nodes:
    train_mask.append(False)
    val_mask.append(False)
    test_mask.append(True)

graph_data.train_mask = torch.tensor(train_mask).cpu()
graph_data.val_mask = torch.tensor(val_mask).cpu()
graph_data.test_mask = torch.tensor(test_mask).cpu()

trainer = pl.Trainer(max_epochs=50)
datamodule = GraphData([graph_data])
trainer.fit(model=model, datamodule=datamodule)

In [129]:
trainer.test(model=model, datamodule=datamodule, verbose=False)

Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 12.81it/s]


[{'step': 50.0,
  'test/auc_weighted': 0.3399044871330261,
  'test/f1_weighted': 0.8851123452186584,
  'test/precision_weighted': 0.8507536053657532,
  'test/recall_weighted': 0.9223629832267761,
  'test/accuracy_weighted': 0.9223629832267761}]

In [130]:
test_auc = trainer.test(model=model, datamodule=datamodule, verbose=False)[0]["test/auc_weighted"]

z, y, y_pred = trainer.predict(model=model, datamodule=datamodule)[0]

print(f'AUC test = {test_auc * 100.:.2f}[%]')

Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 17.53it/s]
AUC test = 33.99[%]


In [131]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [132]:
y_pred.max()

0